In [5]:
# -*- coding: utf-8 -*-
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.optimizers import SGD
import math

In [6]:
# 이미지 데이터 경로 설정
train_path = r'/content/drive/MyDrive/음식'
valid_path = r'/content/drive/MyDrive/음식'

test_path = r'/content/drive/MyDrive/음식'

In [7]:
# 이미지 크기 설정
img_width, img_height = 224, 224

# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 배치 크기 조정
batch_size = 64

train_generator = train_datagen.flow_from_directory(train_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')


Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [8]:
# VGG16 모델 로딩 및 학습 레이어 제거
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
last_layer = vgg16_model.output
last_layer = Flatten()(last_layer)
last_layer = Dense(256, activation='relu')(last_layer)
last_layer = Dropout(0.5)(last_layer)
output_layer = Dense(train_generator.num_classes, activation='softmax')(last_layer)

model = Model(inputs=vgg16_model.input, outputs=output_layer)

58889256/58889256 [==============================] - 0s 0us/step


In [9]:
# 미세 조정을 위한 학습 설정
for layer in model.layers[:15]:
    layer.trainable = False

sgd = SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
# 모델 학습 및 평가
steps_per_epoch = math.floor(train_generator.samples / train_generator.batch_size)
validation_steps = math.floor(valid_generator.samples / valid_generator.batch_size)
test_steps = math.ceil(test_generator.samples / test_generator.batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)

scores = model.evaluate_generator(test_generator, steps=test_steps)
print("Test accuracy: %.2f%%" % (scores[1]*100))

<ipython-input-10-19dca02107cf>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)


Epoch 1/5
6/6 [==============================] - 650s 114s/step - loss: 1.6475 - accuracy: 0.2604 - val_loss: 1.3419 - val_accuracy: 0.3594
Epoch 2/5
6/6 [==============================] - 631s 114s/step - loss: 1.2952 - accuracy: 0.4048 - val_loss: 1.0704 - val_accuracy: 0.5964
Epoch 3/5
6/6 [==============================] - 673s 122s/step - loss: 1.0793 - accuracy: 0.5130 - val_loss: 0.7851 - val_accuracy: 0.6953
Epoch 4/5
6/6 [==============================] - 639s 115s/step - loss: 0.8823 - accuracy: 0.6458 - val_loss: 0.5612 - val_accuracy: 0.8151
Epoch 5/5
6/6 [==============================] - 640s 115s/step - loss: 0.6849 - accuracy: 0.7262 - val_loss: 0.4646 - val_accuracy: 0.8776


<ipython-input-10-19dca02107cf>:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, steps=test_steps)


Test accuracy: 88.00%


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
